전처리

1. 원시데이터 불러오기 
df = pd.read_csv("원시데이터.csv")

2. 사용할 피처 리스트 지정
features = [ ... 위에 나온 28개 변수 ... ]
target = 'HE_BMI'

3. 피처 + 타겟이 모두 존재하는 행만 필터
df_clean = df[features + [target]].dropna()

4. 이상치 제거 (선택사항)
df_clean = df_clean[(df_clean[target] >= 10) & (df_clean[target] <= 60)]

5. 결과 저장
df_clean.to_csv("비만_예측용_정제데이터.csv", index=False)

비만 피쳐
HE_obe_input_features= ['HE_BMI', 'HE_wc', 'HE_wt', 'BO1', 'HE_alt', 'HE_TG', 'HE_sbp2', 'HE_sbp1', 'HE_glu', 'HE_dbp1', 'HE_HbA1c', 'HE_dbp2', 'BO1_1', 'DI1_dg', 'HE_ast', 'HE_crea', 'DI2_dg', 'DI2_pt', 'DI2_pr', 'DI2_2', 'sex', 'DI2_ag', 'DI1_pr', 'DI1_pt', 'DI1_2', 'DI1_ag', 'BO2_1', 'HE_HDL_st2']

실제 구현

In [1]:
!pip install lightgbm shap

Defaulting to user installation because normal site-packages is not writeable
  Using cached lightgbm-4.6.0-py3-none-win_amd64.whl.metadata (17 kB)
  Using cached shap-0.47.2-cp312-cp312-win_amd64.whl.metadata (25 kB)
  Using cached slicer-0.0.8-py3-none-any.whl.metadata (4.0 kB)
Using cached lightgbm-4.6.0-py3-none-win_amd64.whl (1.5 MB)
Using cached shap-0.47.2-cp312-cp312-win_amd64.whl (545 kB)
Using cached slicer-0.0.8-py3-none-any.whl (15 kB)


DEPRECATION: Loading egg at c:\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import shap
import random

In [3]:
# ✅ 1. 데이터 로드 및 전처리

df = pd.read_csv("비만_정제데이터_with_age.csv").dropna()
target_col = 'HE_BMI'
age_col = 'age'
excluded_cols = [target_col, age_col]
features = [col for col in df.columns if col not in excluded_cols]

In [4]:
# ✅ 2. 사용자 입력 샘플

sample = df.sample(1, random_state=random.randint(1, 1000))
user_input = sample[features]
user_age = int(sample[age_col].iloc[0])
user_values = sample.to_dict(orient='records')[0]

In [5]:
# ✅ 3. 모델 학습

X = df[features]
y = df[target_col]
model = lgb.LGBMRegressor()
model.fit(X, y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000825 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1826
[LightGBM] [Info] Number of data points in the train set: 5888, number of used features: 27
[LightGBM] [Info] Start training from score 23.817077


c:\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] 지정된 파일을 찾을 수 없습니다
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

LGBMRegressor()

In [6]:
# ✅ 4. 예측

pred = model.predict(user_input)[0]

In [7]:
# ✅ 5. SHAP 해석

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(user_input)
shap_df = pd.DataFrame({
    'feature': features,
    'shap_value': shap_values[0]
}).sort_values(by='shap_value', key=abs, ascending=False)
top3_features = shap_df.head(3)['feature'].tolist()

In [10]:
print(top3_features)

['HE_wc', 'BO1', 'sex']


In [8]:
# ✅ 6. 유사군 평균 BMI 추정

age_min, age_max = user_age + 3, user_age + 5
similar_group = df[(df[age_col] >= age_min) & (df[age_col] <= age_max)]
group_mean_bmi = similar_group[target_col].mean() if not similar_group.empty else None

In [9]:
# ✅ 7. 최종 출력 문장 생성

output = (
    f"현재 생활습관을 유지하면 3~5년 후 BMI는 약 {pred:.1f}로 예측됩니다.\n"
)
if group_mean_bmi:
    output += f"참고로, 동일한 생활습관을 가진 3~5살 더 많은 유사군의 평균 BMI는 {group_mean_bmi:.1f}였습니다."

print(output.strip())

현재 생활습관을 유지하면 3~5년 후 BMI는 약 19.7로 예측됩니다.
참고로, 동일한 생활습관을 가진 3~5살 더 많은 유사군의 평균 BMI는 23.1였습니다.
